In [99]:
import config
import requests
import json
import ast
import numpy as np
import pandas as pd

In [100]:
# Pass in as arg {0} for the URL, referenced as method_dict[key]
method_dict = {"GetNewsForApp":"ISteamNews/GetNewsForApp/v0002/", 
             "GetGlobalAchievementPercentagesForApp": "ISteamUserStats/GetGlobalAchievementPercentagesForApp/v0002/", 
             "GetGlobalStatsForGame": "ISteamUserStats/GetGlobalStatsForGame/v0001/", 
             "GetPlayerSummaries": "ISteamUser/GetPlayerSummaries/v0002/", 
             "GetFriendList": "ISteamUser/GetFriendList/v0001/", 
             "GetPlayerAchievements": "ISteamUserStats/GetPlayerAchievements/v0001/", 
             "GetUserStatsForGame": "ISteamUserStats/GetUserStatsForGame/v0002/", 
             "GetOwnedGames": "IPlayerService/GetOwnedGames/v0001/", 
             "GetRecentlyPlayedGames": "IPlayerService/GetRecentlyPlayedGames/v0001/", 
             "IsPlayingSharedGame": "IPlayerService/IsPlayingSharedGame/v0001/", 
             "GetSchemaForGame": "ISteamUserStats/GetSchemaForGame/v2/", 
             "GetPlayerBans": "ISteamUser/GetPlayerBans/v1/"
            }

api_method = method_dict["GetOwnedGames"]

# The required api_args are different depending on which method is being called.
# For simplicity's sake, the api_args currently is just steam_id.
# This limits the number of methods that are currently usable.
# Expanding the number of usable methods is not a pressing issue, but will be addressed after minimum viable functionality is achieved.

steam_id = "&steamid={0}".format(config.steam_user)
api_args = steam_id

fetch_url = "http://api.steampowered.com/{0}?key={1}{2}".format(api_method, config.api_key, api_args)

r = requests.get(fetch_url)
returned_values = r.json()
print(json.dumps(returned_values, sort_keys=True, indent=4))

{
    "response": {
        "game_count": 224,
        "games": [
            {
                "appid": 10,
                "playtime_forever": 0
            },
            {
                "appid": 20,
                "playtime_forever": 0
            },
            {
                "appid": 30,
                "playtime_forever": 0
            },
            {
                "appid": 40,
                "playtime_forever": 0
            },
            {
                "appid": 50,
                "playtime_forever": 0
            },
            {
                "appid": 60,
                "playtime_forever": 0
            },
            {
                "appid": 70,
                "playtime_forever": 1326
            },
            {
                "appid": 130,
                "playtime_forever": 0
            },
            {
                "appid": 220,
                "playtime_forever": 1434
            },
            {
                "appid": 240,
                "p

In [101]:
# Checking that I can slice from the JSON data
print(json.dumps(returned_values["response"]["games"][6]["appid"], sort_keys=True, indent=4))

70


In [102]:
# Store total game count in variable
game_count = returned_values["response"]["game_count"]
print("Number of games owned: " + str(game_count))

# Create empty list for counting games played
my_list = []

# Identify each game with playtime over 0 minutes, and add to my_list
for i in returned_values["response"]["games"][0:-1]:
    if i["playtime_forever"] > 0:
        my_list.append(i)

# Count number of games played
game_played = len(my_list)
print("Number of games played: " + str(game_played))

# Calculate games played as a percentage of games owned
game_percent = game_played/game_count
print("Percentage of owned games that have been played: " + str(game_percent))

Number of games owned: 224
Number of games played: 126
Percentage of owned games that have been played: 0.5625


In [103]:
# Retrieving top ten most played games 
sorted_values = json.dumps(sorted(returned_values["response"]["games"], key=lambda i: i["playtime_forever"], reverse=True), indent=4)
top_ten = ast.literal_eval(sorted_values)[:9]

for i in top_ten:
    print(i)

{'appid': 49520, 'playtime_forever': 11179}
{'appid': 730, 'playtime_2weeks': 304, 'playtime_forever': 6834}
{'appid': 10190, 'playtime_forever': 4169}
{'appid': 28050, 'playtime_forever': 3331}
{'appid': 12210, 'playtime_forever': 3014}
{'appid': 241600, 'playtime_forever': 2551}
{'appid': 32370, 'playtime_forever': 2230}
{'appid': 8980, 'playtime_forever': 2116}
{'appid': 9480, 'playtime_forever': 2102}
